In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

print(tf.version.VERSION)
print(keras.__version__)

!wget "https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/model_config.py" -P model_config -nc
!wget "https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/utility_func.py" -P utility_func -nc
import sys
sys.path.append("model_config")
sys.path.append("utility_func")

from utility_func import *
from model_config import *

2.7.0
2.7.0
--2022-02-11 08:13:32--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/model_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6054 (5.9K) [text/plain]
Saving to: ‘model_config/model_config.py’

model_config.py     100%[===================>]   5.91K  --.-KB/s    in 0s      

2022-02-11 08:13:32 (48.6 MB/s) - ‘model_config/model_config.py’ saved [6054/6054]

--2022-02-11 08:13:32--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/utility_func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, await

In [3]:
!wget "https://github.com/ibrahimerdem/application_storage/raw/main/ecommerce_dataset.zip"
!unzip "ecommerce_dataset.zip"
data = pd.read_csv("ecommerce_dataset.csv")

--2022-02-11 08:13:37--  https://github.com/ibrahimerdem/application_storage/raw/main/ecommerce_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/ecommerce_dataset.zip [following]
--2022-02-11 08:13:37--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/ecommerce_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14653198 (14M) [application/zip]
Saving to: ‘ecommerce_dataset.zip’

ecommerce_dataset.z 100%[===================>]  13.97M  --.-KB/s    in 0.1s    

2022-02-11 08:13:37 (122 MB/s) - ‘ecommerce_dataset.zip’ 

# Raw Data Processing

In [4]:
df = data.copy()

In [5]:
df = df[df["status"]=="complete"]
df = df.iloc[:, [0, 2, 20, 3, 8, 4, 5, 17]]
df.rename(columns={"item_id":"t_id",
                   "created_at":"t_date",
                   "Customer ID":"c_id",
                   "sku":"item_name",
                   "category_name_1":"item_category",
                   "qty_ordered":"amount",
                   "Customer Since":"c_since"}, inplace=True)
df["t_date"] = df["t_date"].astype("datetime64")
df["c_since"] = df["c_since"].astype("datetime64")

c_freq = 1
p_freq = 2
m_freq = 1000
t_freq = 2500
modeling_date = "2018-04-01"

df_ =  df[df["t_date"]<datetime.strptime(modeling_date, "%Y-%m-%d")]
print(df_.groupby("c_id")["t_id"].count().describe())
print(df_.groupby("item_name")["t_id"].count().describe())

freq = pd.DataFrame(df_["item_name"].value_counts())
f_items = freq[freq["item_name"]>p_freq].index
df = df[df["item_name"].isin(f_items)]
print("after first filter")
print("#customers:", df["c_id"].nunique(),
      "#items", df["item_name"].nunique())

df_ =  df[df["t_date"]<datetime.strptime(modeling_date, "%Y-%m-%d")]
freq = pd.DataFrame(df_["c_id"].value_counts())
f_cust = freq[freq["c_id"]>c_freq].index
df = df[df["c_id"].isin(f_cust)]

fcat = pd.DataFrame(df["item_category"].value_counts())
f_cat = fcat.loc[fcat["item_category"]<t_freq, ["item_category"]]
df.loc[df["item_category"].isin(f_cat.index), "item_category"] = "other"

print("\nafter second filter")
mind = df["t_date"].min()
maxd = df["t_date"].max()
print("#customers:", df["c_id"].nunique(),
      "#items", df["item_name"].nunique())

print("\ndate between:", mind.day, mind.month, mind.year,
      "-", maxd.day, maxd.month, maxd.year)
print("#transactions:", df["t_id"].nunique())
print("#product-categories:", df["item_category"].nunique())

df.loc[df["price"]==0, "price"] = 1
df["payment"] = df["price"]*df["amount"]
df.drop(columns=["amount"], inplace=True)

df["month"] = df["t_date"].dt.month
df["month"] = df["month"].astype("int64")

df["dayofweek"] = df["t_date"].dt.dayofweek
df["dayofweek"] = df["dayofweek"].astype("int64") + 1

df["recency"] = datetime.strptime(modeling_date, "%Y-%m-%d") - df["t_date"]
df["recency"] = df["recency"].dt.days

df["c_since"] = datetime.strptime(modeling_date, "%Y-%m-%d") - df["c_since"]
df["c_since"] = df["c_since"].dt.days

cust = df["c_id"].unique()
c_size = len(cust)
cust2code = {}
code2cust = {}
for num, c in enumerate(cust):
    cust2code[c] = num+1
    code2cust[num+1] = c

df["c_id"] = df["c_id"].map(cust2code)

items = df["item_name"].unique()
i_size = len(items)
item2code = {}
code2item = {}
for num, item in enumerate(items):
    item2code[item] = num+1
    code2item[num+1] = item

df["item_id"] = df["item_name"].map(item2code)

cats = df["item_category"].unique()
c_size = len(cats)
cat2code = {}
code2cat = {}
for num, cat in enumerate(cats):
    cat2code[cat] = num+1
    code2cat[num+1] = cat

df["category"] = df["item_category"].map(cat2code)
df = df.drop(columns=["item_name", "item_category"])

print("max recency:", df["recency"].max())
print("max price:", df["price"].max())
print("max total-payment:", df["payment"].max())
print("max c_since:", df["c_since"].max())

df["recency"] = pd.cut(df[df["recency"]>0]["recency"],
                       bins=100,
                       labels=list(range(1,101)))
df["c_since"] = pd.cut(df[df["c_since"]>0]["c_since"],
                       bins=100,
                       labels=list(range(1,101)))
df["payment"] = pd.cut(df["payment"],
                       bins=1000,
                       labels=list(range(1,1001)))
df["price"] = pd.cut(df["price"],
                     bins=500,
                     labels=list(range(1,501)))

df.to_csv("data_set.csv",
          sep=";",
          index=False)

count    65089.000000
mean         3.486887
std         16.744492
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       1657.000000
Name: t_id, dtype: float64
count    42463.000000
mean         5.344841
std         27.581032
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       2863.000000
Name: t_id, dtype: float64
after first filter
#customers: 58340 #items 13026

after second filter
#customers: 25939 #items 12907

date between: 1 7 2016 - 30 7 2018
#transactions: 160046
#product-categories: 13
max recency: 639
max price: 111856.0
max total-payment: 8944000.0
max c_since: 639


# Processed Data

In [34]:
processed_data = pd.read_csv("data_set.csv",
                             sep=";")

# Time-Basis Application

In [35]:
df_a = processed_data.copy()

model_start = "07-01-2016"
model_end = "04-01-2018"
test_start = "04-01-2018"
test_end = "07-01-2018"

df_a["t_date"] = df_a["t_date"].astype("datetime64")
df_base = df_a[(df_a["t_date"]>=model_start)&(df_a["t_date"]<model_end)]

pro_max = df_base["item_id"].astype("int64").max()
rec_max = df_base["recency"].astype("int64").max()
pay_max = df_base["payment"].astype("int64").max()
cat_max = df_base["category"].astype("int64").max()
mon_max = df_base["month"].astype("int64").max()
day_max = df_base["dayofweek"].astype("int64").max()
pri_max = df_base["price"].astype("int64").max()
sin_max = df_base["c_since"].astype("int64").max()

n_padded = 24
batch_size = 256
train_size = 0.9

hold_df = df_a.loc[(df_a["t_date"]>=test_start)&(df_a["t_date"]<test_end),
                   ["c_id", "item_id"]]

items = df_base["item_id"].unique()
hold_df = hold_df[hold_df["item_id"].isin(items)]
hold_c = hold_df["c_id"].unique()

train = df_base[~(df_base["c_id"].isin(hold_c))]
test_data = df_base[df_base["c_id"].isin(hold_c)]

train_c = train["c_id"].unique()
random_selection = np.random.rand(len(train_c)) <= train_size
val_c = train_c[~random_selection]
val_data = train[train["c_id"].isin(val_c)]
train_data = train[~(train["c_id"].isin(val_c))]

train_data["recency"] = train_data["recency"].astype("int")
test_data["recency"] = test_data["recency"].astype("int")
val_data["recency"] = val_data["recency"].astype("int")

print("train customers:", train_data["c_id"].nunique())
print("validation customers:", val_data["c_id"].nunique())
print("test customers:", test_data["c_id"].nunique())
print(test_data.groupby("c_id")["t_id"].count().describe())

train customers: 22950
validation customers: 2600
test customers: 389
count    389.000000
mean      23.082262
std       60.908444
min        2.000000
25%        4.000000
50%        7.000000
75%       21.000000
max      855.000000
Name: t_id, dtype: float64


In [36]:
train_set = make_data_dict(train_data, n_padded)
val_set = make_data_dict(val_data, n_padded)

train_set = create_batch_data(train_set, batch=batch_size)
val_set = create_batch_data(val_set, batch=batch_size)

## Model

In [ ]:
n_dim_pro = 128

input_item = layers.Input(shape=(n_padded,),
                          name="item_ids",
                          dtype=tf.int32)
encoding_padding_mask = tf.math.logical_not(tf.math.equal(input_item, 0))
embedding_item = layers.Embedding(input_dim=pro_max+1,
                                  output_dim=n_dim_pro)(input_item)

input_price = layers.Input(shape=(n_padded, pri_max),
                           name="price",
                           dtype=tf.float32)

input_category = layers.Input(shape=(n_padded, cat_max),
                              name="category",
                              dtype=tf.float32)

input_recency = layers.Input(shape=(n_padded, rec_max),
                             name="recency",
                             dtype=tf.float32)

input_month = layers.Input(shape=(n_padded, mon_max),
                           name="month",
                           dtype=tf.float32)

input_dayofweek = layers.Input(shape=(n_padded, day_max),
                               name="dayofweek",
                               dtype=tf.float32)

input_payment = layers.Input(shape=(n_padded, pay_max),
                             name="payment",
                             dtype=tf.float32)

input_since = layers.Input(shape=(sin_max,),
                           name="c_since",
                           dtype=tf.float32)
r = layers.RepeatVector(n_padded)(input_since)

input_concat = layers.Concatenate()(
    [embedding_item,
     input_category,
     input_recency,
     input_month,
     input_dayofweek,
     input_payment,
     input_price,
     ])

input_concat = layers.BatchNormalization()(input_concat)

x = layers.LSTM(units=400, 
                return_sequences=True)(input_concat)

x = layers.Dropout(rate=0.8)(x)
x = layers.BatchNormalization()(x)

x = layers.Concatenate()([x, r])

z = layers.Attention(use_scale=False,
                     causal=True)(inputs=[x, x],
                                  mask=[encoding_padding_mask,
                                        encoding_padding_mask])       

output = layers.Dense(pro_max+1)(z)

model = keras.Model([input_item,
                     input_category,
                     input_recency,
                     input_month,
                     input_dayofweek,
                     input_payment,
                     input_price,
                     input_since], output)
model.compile(loss=loss_function,
              optimizer=keras.optimizers.Adam(0.0001),
              metrics=["sparse_categorical_accuracy"])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 item_ids (InputLayer)          [(None, 24)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 24, 128)      1651712     ['item_ids[0][0]']               
                                                                                                  
 category (InputLayer)          [(None, 24, 13)]     0           []                               
                                                                                                  
 recency (InputLayer)           [(None, 24, 100)]    0           []                               
                                                                                              

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   verbose=1,
                                   patience=3)

model.fit(train_set,
          epochs=50,
          validation_data=val_set,
          callbacks=[es])

Epoch 1/50
90/90 [==============================] - 32s 274ms/step - loss: 0.3180 - sparse_categorical_accuracy: 0.0099 - val_loss: 0.3044 - val_sparse_categorical_accuracy: 0.0294
Epoch 2/50
90/90 [==============================] - 25s 276ms/step - loss: 0.3026 - sparse_categorical_accuracy: 0.0225 - val_loss: 0.2975 - val_sparse_categorical_accuracy: 0.0283
Epoch 3/50
90/90 [==============================] - 25s 277ms/step - loss: 0.2929 - sparse_categorical_accuracy: 0.0268 - val_loss: 0.2904 - val_sparse_categorical_accuracy: 0.0295
Epoch 4/50
90/90 [==============================] - 25s 282ms/step - loss: 0.2876 - sparse_categorical_accuracy: 0.0360 - val_loss: 0.2842 - val_sparse_categorical_accuracy: 0.0448
Epoch 5/50
90/90 [==============================] - 27s 298ms/step - loss: 0.2833 - sparse_categorical_accuracy: 0.0487 - val_loss: 0.2777 - val_sparse_categorical_accuracy: 0.0722
Epoch 6/50
90/90 [==============================] - 27s 298ms/step - loss: 0.2788 - sparse_cate

## Evaluation

In [ ]:
test_set = make_data_dict(test_data, n_padded, True)
test_set = create_batch_data(test_set, test=True, batch=batch_size)

In [ ]:
y_pred = model.predict(test_set)

customer_list = test_data["c_id"].unique()

main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    ret = y_pred[i][-1].argsort()[::-1][:k]
    rlist = ",".join([str(i) for i in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(i) for i in x]),
     "hit": "max",
     "r_list": "last"})

print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.2185
Recall@20 : 0.1561
MAP@20 : 0.0925
NDCG@20 : 0.134
0    304
1     85
Name: hit, dtype: int64
0    0.781491
1    0.218509
Name: hit, dtype: float64


### Baseline

#### Last-Items (Random)

In [ ]:
customer_list = test_data["c_id"].unique()

main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    re = list(test_data.loc[test_data["c_id"]==c, "item_id"].unique())
    if len(re) > k:
        ret = np.random.choice(re, k, replace=False)
    else:
        ret = list()
        while len(re)>0:
            ri = re.pop()
            if ri not in ret:
                ret.append(ri)
    rlist = ",".join([str(i) for i in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist 
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(y) for y in x]),
     "hit": "max",
     "r_list": "last"})

print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.1517
Recall@20 : 0.1049
MAP@20 : 0.0749
NDCG@20 : 0.1031
0    330
1     59
Name: hit, dtype: int64
0    0.848329
1    0.151671
Name: hit, dtype: float64


#### Last-Items

In [ ]:
item_list = test_data.groupby("c_id", as_index=False).agg({"item_id":lambda i: list(i)})
customer_list = test_data["c_id"].unique()
main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    re = item_list.loc[item_list["c_id"]==c, "item_id"].values[0]
    ret = list()
    while len(re)>0:
        ri = re.pop()
        if ri not in ret:
            ret.append(ri)
    ret = ret[:k]
    rlist = ",".join([str(x) for x in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(y) for y in x]),
     "hit": "max",
     "r_list": "last"})


print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.1697
Recall@20 : 0.1118
MAP@20 : 0.1037
NDCG@20 : 0.1328
0    323
1     66
Name: hit, dtype: int64
0    0.830334
1    0.169666
Name: hit, dtype: float64


# Last-Event Application

In [ ]:
df_b = processed_data.copy()

model_start = "07-01-2016"
model_end = "04-01-2018"

df_b["t_date"] = df_b["t_date"].astype("datetime64")
df_base = df_b[(df_b["t_date"]>=model_start)&(df_b["t_date"]<model_end)]

pro_max = df_base["item_id"].astype("int64").max()
rec_max = df_base["recency"].astype("int64").max()
pay_max = df_base["payment"].astype("int64").max()
cat_max = df_base["category"].astype("int64").max()
met_max = df_base["method"].astype("int64").max()
mon_max = df_base["month"].astype("int64").max()
day_max = df_base["dayofweek"].astype("int64").max()
pri_max = df_base["price"].astype("int64").max()
sin_max = df_base["c_since"].astype("int64").max()

n_padded = 24
batch_size = 256
test_size = 0.025
val_size = 0.1

all_c = df_base["c_id"].unique()
random_selection = np.random.rand(len(all_c)) <= 1-test_size
test_c = all_c[~random_selection]

train = df_base[~(df_base["c_id"].isin(test_c))]
test = df_base[df_base["c_id"].isin(test_c)]

train_c = train["c_id"].unique()
random_selection = np.random.rand(len(train_c)) <= 1-val_size
val_c = train_c[~random_selection]
val_data = train[train["c_id"].isin(val_c)]
train_data = train[~(train["c_id"].isin(val_c))]

lasts = test.groupby("c_id", as_index=False).last()
hold_df = lasts[["c_id", "item_id"]]
last_tr = lasts["t_id"].unique()
test_data = test[~(test["t_id"].isin(last_tr))]

train_data["recency"] = train_data["recency"].astype("int")
test_data["recency"] = test_data["recency"].astype("int")
val_data["recency"] = val_data["recency"].astype("int")

print("train customers:", train_data["c_id"].nunique())
print("validation customers:", val_data["c_id"].nunique())
print("test customers:", test_data["c_id"].nunique())

train customers: 22763
validation customers: 2527
test customers: 649


In [ ]:
train_set = make_data_dict(train_data, n_padded)
val_set = make_data_dict(val_data, n_padded)

train_set = create_batch_data(train_set, batch=batch_size)
val_set = create_batch_data(val_set, batch=batch_size)

## Model

In [ ]:
n_dim_pro = 128

input_item = layers.Input(shape=(n_padded,),
                          name="item_ids",
                          dtype=tf.int32)
encoding_padding_mask = tf.math.logical_not(tf.math.equal(input_item, 0))
embedding_item = layers.Embedding(input_dim=pro_max+1,
                                  output_dim=n_dim_pro)(input_item)

input_price = layers.Input(shape=(n_padded, pri_max),
                           name="price",
                           dtype=tf.float32)

input_category = layers.Input(shape=(n_padded, cat_max),
                              name="category",
                              dtype=tf.float32)

input_recency = layers.Input(shape=(n_padded, rec_max),
                             name="recency",
                             dtype=tf.float32)

input_month = layers.Input(shape=(n_padded, mon_max),
                           name="month",
                           dtype=tf.float32)

input_dayofweek = layers.Input(shape=(n_padded, day_max),
                               name="dayofweek",
                               dtype=tf.float32)

input_payment = layers.Input(shape=(n_padded, pay_max),
                             name="payment",
                             dtype=tf.float32)

input_since = layers.Input(shape=(sin_max,),
                           name="c_since",
                           dtype=tf.float32)
r = layers.RepeatVector(n_padded)(input_since)

input_concat = layers.Concatenate()(
    [embedding_item,
     input_category,
     input_recency,
     input_month,
     input_dayofweek,
     input_payment,
     input_price,
     ])

input_concat = layers.BatchNormalization()(input_concat)

x = layers.LSTM(units=400, 
                return_sequences=True)(input_concat)

x = layers.Dropout(rate=0.8)(x)
x = layers.BatchNormalization()(x)

x = layers.Concatenate()([x, r])

z = layers.Attention(use_scale=False,
                     causal=True)(inputs=[x, x],
                                  mask=[encoding_padding_mask,
                                        encoding_padding_mask])       

output = layers.Dense(pro_max+1)(z)

model = keras.Model([input_item,
                     input_category,
                     input_recency,
                     input_month,
                     input_dayofweek,
                     input_payment,
                     input_price,
                     input_since], output)
model.compile(loss=loss_function,
              optimizer=keras.optimizers.Adam(0.0001),
              metrics=["sparse_categorical_accuracy"])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 item_ids (InputLayer)          [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 128)      1651712     ['item_ids[0][0]']               
                                                                                                  
 category (InputLayer)          [(None, 32, 13)]     0           []                               
                                                                                                  
 recency (InputLayer)           [(None, 32, 100)]    0           []                               
                                                                                              

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   verbose=1,
                                   patience=3)

model.fit(train_set,
          epochs=50,
          validation_data=val_set,
          callbacks=[es])

Epoch 1/50
89/89 [==============================] - 34s 296ms/step - loss: 0.3312 - sparse_categorical_accuracy: 0.0105 - val_loss: 0.3488 - val_sparse_categorical_accuracy: 0.0199
Epoch 2/50
89/89 [==============================] - 26s 296ms/step - loss: 0.3151 - sparse_categorical_accuracy: 0.0220 - val_loss: 0.3411 - val_sparse_categorical_accuracy: 0.0202
Epoch 3/50
89/89 [==============================] - 26s 293ms/step - loss: 0.3055 - sparse_categorical_accuracy: 0.0252 - val_loss: 0.3333 - val_sparse_categorical_accuracy: 0.0215
Epoch 4/50
89/89 [==============================] - 27s 298ms/step - loss: 0.3001 - sparse_categorical_accuracy: 0.0329 - val_loss: 0.3265 - val_sparse_categorical_accuracy: 0.0339
Epoch 5/50
89/89 [==============================] - 26s 294ms/step - loss: 0.2954 - sparse_categorical_accuracy: 0.0456 - val_loss: 0.3195 - val_sparse_categorical_accuracy: 0.0578
Epoch 6/50
89/89 [==============================] - 26s 293ms/step - loss: 0.2909 - sparse_cate

## Evaluation

In [ ]:
test_set = make_data_dict(test_data, n_padded, True)
test_set = create_batch_data(test_set, test=True, batch=batch_size)

In [ ]:
y_pred = model.predict(test_set)

customer_list = test_data["c_id"].unique()

main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    ret = y_pred[i][-1].argsort()[::-1][:k]
    rlist = ",".join([str(i) for i in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(i) for i in x]),
     "hit": "max",
     "r_list": "last"})

print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.327
Recall@20 : 0.327
MAP@20 : 0.1724
NDCG@20 : 0.2226
0    463
1    225
Name: hit, dtype: int64
0    0.672965
1    0.327035
Name: hit, dtype: float64


### Baseline

#### Last-Items (Random)

In [ ]:
customer_list = test_data["c_id"].unique()

main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    re = list(test_data.loc[test_data["c_id"]==c, "item_id"].unique())
    if len(re) > k:
        ret = np.random.choice(re, k, replace=False)
    else:
        ret = list()
        while len(re)>0:
            ri = re.pop()
            if ri not in ret:
                ret.append(ri)
    rlist = ",".join([str(i) for i in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist 
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(y) for y in x]),
     "hit": "max",
     "r_list": "last"})

print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.2065
Recall@20 : 0.2065
MAP@20 : 0.1839
NDCG@20 : 0.1972
0    515
1    134
Name: hit, dtype: int64
0    0.793529
1    0.206471
Name: hit, dtype: float64


#### Last-Items

In [ ]:
item_list = test_data.groupby("c_id", as_index=False).agg({"item_id":lambda i: list(i)})
customer_list = test_data["c_id"].unique()
main_table = hold_df[hold_df["c_id"].isin(customer_list)]
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()
    re = item_list.loc[item_list["c_id"]==c, "item_id"].values[0]
    ret = list()
    while len(re)>0:
        ri = re.pop()
        if ri not in ret:
            ret.append(ri)
    ret = ret[:k]
    rlist = ",".join([str(x) for x in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist
    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(y) for y in x]),
     "hit": "max",
     "r_list": "last"})


print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.2065
Recall@20 : 0.2065
MAP@20 : 0.1862
NDCG@20 : 0.1984
0    515
1    134
Name: hit, dtype: int64
0    0.793529
1    0.206471
Name: hit, dtype: float64
